In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentimental-analysis-for-tweets/sentiment_tweets3.csv


In [2]:
data = pd.read_csv("/kaggle/input/sentimental-analysis-for-tweets/sentiment_tweets3.csv", encoding="utf-8")
print(data.head())

   Index                                 message to examine  \
0    106  just had a real good moment. i missssssssss hi...   
1    217         is reading manga  http://plurk.com/p/mzp1e   
2    220  @comeagainjen http://twitpic.com/2y2lx - http:...   
3    288  @lapcat Need to send 'em to my accountant tomo...   
4    540      ADD ME ON MYSPACE!!!  myspace.com/LookThunder   

   label (depression result)  
0                          0  
1                          0  
2                          0  
3                          0  
4                          0  


In [3]:
data.columns

Index(['Index', 'message to examine', 'label (depression result)'], dtype='object')

In [4]:
msg = data[['message to examine']]
print(msg.head())

                                  message to examine
0  just had a real good moment. i missssssssss hi...
1         is reading manga  http://plurk.com/p/mzp1e
2  @comeagainjen http://twitpic.com/2y2lx - http:...
3  @lapcat Need to send 'em to my accountant tomo...
4      ADD ME ON MYSPACE!!!  myspace.com/LookThunder


In [5]:
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from snowballstemmer import stemmer
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')

def remove_chars(text, del_chars):
    translator = str.maketrans('', '', del_chars)
    return text.translate(translator)

def remove_repeating_char(text):
    return re.sub(r'(.)\1{2,}', r'\1', text)

def cleaningText(text):
      text = re.sub(r'[0-9]+', '', text)

      english_punctuations = string.punctuation

      text = remove_repeating_char(text)
      text = text.replace('\n', ' ') 
      text = text.strip(' ')  
      return text

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
 def tokenizingText(text):
     tokens_list = word_tokenize(text) 
     return tokens_list

 def filteringText(tokens_list):
     listStopwords = set(stopwords.words('english'))
     filtered = []
     for txt in tokens_list:
         if txt not in listStopwords:
             filtered.append(txt)
             tokens_list = filtered
             return tokens_list

def stemmingText(tokens_list):
    tokens_list = [stemmer.stemWord(word) for word in tokens_list]
    return tokens_list

def toSentence(words_list):
     sentence = ' '.join(word for word in words_list)
     return sentence

In [7]:
print(msg.head(20))

                                   message to examine
0   just had a real good moment. i missssssssss hi...
1          is reading manga  http://plurk.com/p/mzp1e
2   @comeagainjen http://twitpic.com/2y2lx - http:...
3   @lapcat Need to send 'em to my accountant tomo...
4       ADD ME ON MYSPACE!!!  myspace.com/LookThunder
5               so sleepy. good times tonight though 
6   @SilkCharm re: #nbn as someone already said, d...
7                  23 or 24ï¿½C possible today. Nice 
8         nite twitterville  workout in the am  -ciao
9     @daNanner Night, darlin'!  Sweet dreams to you 
10                           Good morning everybody! 
11  Finally! I just created my WordPress Blog. The...
12  kisha they cnt get over u til they get out frm...
13  @nicolerichie Yes i remember that band, It was...
14             I really love reflections and shadows 
15  @blueaero ooo it's fantasy?  i like fantasy no...
16  @rokchic28 no probs, I sell nothing other than...
17  @shipovalov &quot;NOKLA 

In [8]:
def preprocess_text(text):
    text = str(text).lower()
    
    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    
    # Remove @ mentions
    text = re.sub(r'@\w+', '', text)
    
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    
    # Remove HTML entities
    text = re.sub(r'&[a-zA-Z]+;', '', text)
    
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', ' ', text)

    # Remove extra whitespace and repeated characters
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    
    # Remove digits
    text = re.sub(r'\d+', '', text)
    
    # Strip leading/trailing whitespace
    text = text.strip()
    
    return text

In [9]:
def stem_text(text):
    stemmer = PorterStemmer()
    
    # Split into words
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words if len(word) > 1]
    
    return ' '.join(stemmed_words)

In [10]:
def clean_and_stem_data(df, text_column='message to examine'):
    # Create a copy to avoid modifying original data
    df_clean = df.copy()
    
    # Apply preprocessing
    df_clean['cleaned_text'] = df_clean[text_column].apply(preprocess_text)
    
    # Apply stemming
    df_clean['stemmed_text'] = df_clean['cleaned_text'].apply(stem_text)
    
    return df_clean

In [11]:
df_processed = clean_and_stem_data(msg)
print("Original vs Cleaned vs Stemmed Text:")
print("=" * 80)

Original vs Cleaned vs Stemmed Text:


In [12]:
for i in range(len(df_processed)):
    print(f"\nIndex {i}:")
    print(f"Original: {df_processed.iloc[i]['message to examine']}")
    print(f"Cleaned:  {df_processed.iloc[i]['cleaned_text']}")
    print(f"Stemmed:  {df_processed.iloc[i]['stemmed_text']}")


Index 0:
Original: just had a real good moment. i missssssssss him so much, 
Cleaned:  just had a real good moment i miss him so much
Stemmed:  just had real good moment miss him so much

Index 1:
Original: is reading manga  http://plurk.com/p/mzp1e
Cleaned:  is reading manga
Stemmed:  is read manga

Index 2:
Original: @comeagainjen http://twitpic.com/2y2lx - http://www.youtube.com/watch?v=zoGfqvh2ME8 
Cleaned:  
Stemmed:  

Index 3:
Original: @lapcat Need to send 'em to my accountant tomorrow. Oddly, I wasn't even referring to my taxes. Those are supporting evidence, though. 
Cleaned:  need to send em to my accountant tomorrow oddly i wasn t even referring to my taxes those are supporting evidence though
Stemmed:  need to send em to my account tomorrow oddli wasn even refer to my tax those are support evid though

Index 4:
Original: ADD ME ON MYSPACE!!!  myspace.com/LookThunder
Cleaned:  add me on myspace myspace com lookthunder
Stemmed:  add me on myspac myspac com lookthund

Index 

In [13]:
print(f"\n\nSummary:")
print(f"Total messages processed: {len(df_processed)}")
print(f"Average original length: {df_processed['message to examine'].str.len().mean():.1f} characters")
print(f"Average cleaned length: {df_processed['cleaned_text'].str.len().mean():.1f} characters")
print(f"Average stemmed length: {df_processed['stemmed_text'].str.len().mean():.1f} characters")




Summary:
Total messages processed: 10314
Average original length: 92.6 characters
Average cleaned length: 74.3 characters
Average stemmed length: 65.8 characters


The cleaning of the messages resulted in a significant reduction in their length, from 92.6 to 74.3 characters on average. 